In [1]:
!cp -r ../input/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./
%config Completer.use_jedi = False
%load_ext Cython
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout

In [2]:
URM_all_dataframe = pd.read_csv('/kaggle/input/urm-true-binary/URM_True_Binary.csv')
URM_all_dataframe
URM_all = sps.coo_matrix((URM_all_dataframe["Data"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1554640 stored elements in Compressed Sparse Row format>

In [3]:
URM_train_dataframe = pd.read_csv('/kaggle/input/urm-split/Train_df.csv')
URM_train_dataframe
URM_train = sps.coo_matrix((URM_train_dataframe["Data"].values, 
                          (URM_train_dataframe["UserID"].values, URM_train_dataframe["ItemID"].values)))
URM_train = URM_train.tocsr() # to obtain fast access to rows (users)
URM_train

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 1243712 stored elements in Compressed Sparse Row format>

In [4]:
URM_val_dataframe = pd.read_csv('/kaggle/input/urm-split/Test_df.csv')
URM_val_dataframe
URM_valid = sps.coo_matrix((URM_val_dataframe["Data"].values, 
                          (URM_val_dataframe["UserID"].values, URM_val_dataframe["ItemID"].values)))
URM_valid = URM_valid.tocsr() # to obtain fast access to rows (users)
URM_valid

<41629x24507 sparse matrix of type '<class 'numpy.int64'>'
	with 310928 stored elements in Compressed Sparse Row format>

In [5]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 323 ( 0.8%) Users that have less than 1 test interactions


In [6]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender

#try a SLIM BPR model
recommender_class = P3alphaRecommender
import os

output_folder_path = "result_experiments/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 500
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [7]:
from skopt.space import Real, Integer, Categorical
#RP3 beta
#to tuning hyperparam are typical of ML models to drive the learning process

hyperparameters_range_dictionary = {
    "topK": Integer(30,2000),
    "alpha": Real(low=0, high = 2, prior="uniform"),
    "implicit": Categorical([True,False]),
    "normalize_similarity": Categorical([True, False]),
    
}
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

#create a bayesian optimizer object, we pass the recommender and the evaluator
hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_valid)
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
#provide data needed to create instance of model (one on URM_train, the other on URM_all)
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_all],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [8]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 464, 'alpha': 0.9416409634789804, 'implicit': True, 'normalize_similarity': False}
P3alphaRecommender: Similarity column 24507 (100.0%), 1328.02 column/sec. Elapsed time 18.45 sec
EvaluatorHoldout: Processed 41306 (100.0%) in 31.66 sec. Users per second: 1305
SearchBayesianSkopt: New best config found. Config 0: {'topK': 464, 'alpha': 0.9416409634789804, 'implicit': True, 'normalize_similarity': False} - results: PRECISION: 0.0591827, PRECISION_RECALL_MIN_DEN: 0.0800488, RECALL: 0.0691978, MAP: 0.0292890, MAP_MIN_DEN: 0.0388120, MRR: 0.1841557, NDCG: 0.0824295, F1: 0.0637996, HIT_RATE: 0.3698252, ARHR_ALL_HITS: 0.2298290, NOVELTY: 0.0041799, AVERAGE_POPULARITY: 0.2245688, DIVERSITY_MEAN_INTER_LIST: 0.9142714, DIVERSITY_HERFINDAHL: 0.9914249, COVERAGE_ITEM: 0.3975599, COVERAGE_ITEM_HIT: 0.0428449, ITEMS_IN_GT: 0.9983678, COVERAGE_USER: 0.9922410, COVERAGE_USER_HIT: 0.3669557, USER